<a href="https://colab.research.google.com/github/gabrielapb36/OurWorldData/blob/main/1_Crisis_fiscales_en_grandes_econom%C3%ADas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Figura 1. Evolución del déficit del gobierno general, en el periodo 2016-2030 (% del PIB).
Nota. El déficit refleja el saldo fiscal neto. Proyecciones 2025–2030 corresponden al escenario base del IMF. Elaboración del Ceplan a partir de la base de datos de (IMF, 2025).
"""
import pandas as pd
import matplotlib.pyplot as plt # Keep import in case user wants to plot later

# === 1. Cargar los datos ===
ruta = "dataset_2025-10-19T19_30_17.880841389Z_DEFAULT_INTEGRATION_IMF.FAD_FM_5.0.0.csv"
df = pd.read_csv(ruta)

# === 2. Seleccionar países del G7 y Perú ===
paises_seleccionados = ['United States', 'United Kingdom', 'Canada', 'France', 'Germany', 'Italy', 'Japan', 'Peru']
df_seleccionado = df[df['COUNTRY'].isin(paises_seleccionados)].copy() # Use a new DataFrame to keep original df intact

# === 3. Pivot the dataframe to convert year columns to rows ===
df_melted = df_seleccionado.melt(id_vars=['DATASET', 'SERIES_CODE', 'OBS_MEASURE', 'COUNTRY', 'INDICATOR'],
             var_name='Year',
             value_name='Value')

# Convert 'Year' to numeric, coercing errors to NaN
df_melted['Year'] = pd.to_numeric(df_melted['Year'], errors='coerce')

# Drop rows where 'Year' is NaN (non-year columns in melt)
df_melted.dropna(subset=['Year'], inplace=True)

# === 4. Procesar cada indicador y generar tabla pivotada ===
for indicador in df_melted['INDICATOR'].unique():
    print(f"### Procesando Indicador: {indicador}") # Added print for debugging

    # Filter data for the current indicator
    df_indicador = df_melted[df_melted['INDICATOR'] == indicador].copy()

    # Added print and display for debugging 'Gross debt'
    if 'Gross debt, General government, Percent of GDP' in indicador:
        print(f"Debugging data for '{indicador}':")
        display(df_indicador.head())
        print("-" * 30)


    # Calculate G7 average for the current indicator (still only for G7 countries)
    g7_avg_indicador = (
        df_indicador[df_indicador['COUNTRY'].isin(g7)].groupby('Year')['Value'] # Calculate average only for G7
        .mean()
        .reset_index()
        .assign(**{'COUNTRY': 'G7 Average'})
    )

    # Combine country data and G7 average
    df_final_indicador = pd.concat([df_indicador, g7_avg_indicador], ignore_index=True)

    # Pivot the table
    tabla_pivotada_indicador = df_final_indicador.pivot(index='Year', columns='COUNTRY', values='Value')

    # Display the pivoted table
    print(f"### Indicador: {indicador}\n") # Moved this print to ensure it appears before the table
    display(tabla_pivotada_indicador)

    print("-" * 50) # Separator between tables

### Procesando Indicador: Gross debt, General government, Percent of GDP
Debugging data for 'Gross debt, General government, Percent of GDP':


,DATASET,SERIES_CODE,OBS_MEASURE,COUNTRY,INDICATOR,Year,Value
48,IMF.FAD:FM(5.0.0),CAN.G63G_S13_POGDP_PT.A,OBS_VALUE,Canada,"Gross debt, General government, Percent of GDP",2016.0,92.396824
50,IMF.FAD:FM(5.0.0),DEU.G63G_S13_POGDP_PT.A,OBS_VALUE,Germany,"Gross debt, General government, Percent of GDP",2016.0,68.301457
54,IMF.FAD:FM(5.0.0),FRA.G63G_S13_POGDP_PT.A,OBS_VALUE,France,"Gross debt, General government, Percent of GDP",2016.0,98.126237
57,IMF.FAD:FM(5.0.0),JPN.G63G_S13_POGDP_PT.A,OBS_VALUE,Japan,"Gross debt, General government, Percent of GDP",2016.0,232.42123
59,IMF.FAD:FM(5.0.0),ITA.G63G_S13_POGDP_PT.A,OBS_VALUE,Italy,"Gross debt, General government, Percent of GDP",2016.0,134.239783


------------------------------
### Indicador: Gross debt, General government, Percent of GDP



COUNTRY,Canada,France,G7 Average,Germany,Italy,Japan,Peru,United Kingdom,United States
Year,,,,,,,,,
2016.0,92.396824,98.126237,117.2416,68.301457,134.239783,232.42123,23.931762,87.82519,107.380475
2017.0,90.942246,98.722292,115.972984,63.987807,133.740901,231.320766,24.799424,86.722814,106.37406
2018.0,90.776119,98.490127,115.796094,60.757536,134.198344,232.380301,25.688585,86.345145,107.625087
2019.0,90.210165,98.104314,115.954782,58.681897,133.898057,236.38106,26.501313,85.657192,108.750785
2020.0,118.065576,114.906594,136.015736,68.036758,154.379158,258.368864,34.289418,105.840115,132.513083
2021.0,112.617309,112.785724,131.856815,67.990897,145.812393,253.652035,35.543706,105.130743,125.0086
2022.0,104.207384,111.365574,126.474979,64.442108,138.345146,248.245999,33.476812,99.614198,119.104446
2023.0,107.710724,109.622557,125.000547,62.382309,134.637396,240.461035,32.430899,100.354236,119.835568
2024.0,111.302178,113.148106,126.128313,63.465397,135.32628,236.107529,32.150583,101.226943,122.321755


--------------------------------------------------
### Procesando Indicador: Net lending (+) / net borrowing (-), General government, Percent of GDP
### Indicador: Net lending (+) / net borrowing (-), General government, Percent of GDP



COUNTRY,Canada,France,G7 Average,Germany,Italy,Japan,Peru,United Kingdom,United States
Year,,,,,,,,,
2016.0,-0.453164,-3.762716,-2.393528,1.134104,-2.404661,-3.59961,-2.118061,-3.312287,-4.356365
2017.0,-0.112443,-3.364914,-2.147589,1.33722,-2.508865,-3.098813,-2.78861,-2.491114,-4.794193
2018.0,0.359891,-2.321672,-1.763858,1.883123,-2.191992,-2.471408,-1.944371,-2.270079,-5.334866
2019.0,-0.017289,-2.391688,-1.979159,1.327263,-1.450112,-3.044644,-1.367851,-2.472245,-5.805397
2020.0,-10.949067,-8.930231,-10.001002,-4.379695,-9.378789,-9.075077,-8.161666,-13.172942,-14.121212
2021.0,-3.05712,-6.583672,-6.710557,-3.16728,-8.879364,-6.127706,-2.49062,-7.724818,-11.433938
2022.0,0.555641,-4.737402,-3.812887,-1.908512,-8.109313,-4.214704,-1.38249,-4.555602,-3.720318
2023.0,0.071954,-5.439608,-4.459149,-2.49446,-7.238656,-2.260709,-2.778846,-6.054216,-7.79835
2024.0,-2.013998,-5.770655,-4.172004,-2.663336,-3.446203,-1.529777,-3.538906,-5.748742,-8.031315


--------------------------------------------------
### Procesando Indicador: Primary net lending (+) / net borrowing (-), General government, Percent of GDP
### Indicador: Primary net lending (+) / net borrowing (-), General government, Percent of GDP



COUNTRY,Canada,France,G7 Average,Germany,Italy,Japan,Peru,United Kingdom,United States
Year,,,,,,,,,
2016.0,0.130731,-1.928875,-0.712258,2.066406,1.338471,-2.531575,-1.201395,-1.686847,-2.374115
2017.0,0.089366,-1.634656,-0.555744,2.170165,1.096192,-2.163693,-1.803055,-0.666752,-2.780829
2018.0,0.462455,-0.606947,-0.229886,2.606617,1.2895,-1.67808,-0.826029,-0.581656,-3.101088
2019.0,0.103606,-0.918211,-0.582663,1.918084,1.737906,-2.364087,-0.212554,-1.028371,-3.527567
2020.0,-10.45477,-7.66498,-8.668332,-3.916277,-6.10145,-8.435176,-6.784027,-12.033833,-12.07184
2021.0,-3.699911,-5.21718,-5.35747,-2.703197,-5.596014,-5.542227,-1.213589,-5.580249,-9.163513
2022.0,0.172399,-2.860836,-1.989001,-1.348453,-4.176624,-3.809804,-0.04803,-0.912434,-0.987254
2023.0,0.287578,-3.740084,-2.772564,-1.850282,-3.737936,-2.047574,-1.337536,-3.650175,-4.669475
2024.0,-1.874398,-3.874177,-2.400488,-1.802214,0.293452,-1.145694,-2.111646,-3.785949,-4.614433


--------------------------------------------------
